This notebook contains script that instantiates 'labelImg' window for creating bbox and convert those for retinnanet algorithm to consume. Note that image size is assumed to be 640 x 640

In [1]:
import xml.etree.ElementTree as ET
import numpy as np
import glob
import os
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
from six import BytesIO
import re

2023-06-06 12:32:54.778329: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#images and xml are to be stored in following directory
#data_dir = "..//data"
data_dir = "../image_preprocessing/output_data/"

In [3]:
#check naming convention of image data. it is expected to be 3 digit numeral. this will help in reducing error moving forward
image_file_names = []
os.chdir(data_dir)
for file in glob.glob("*.jpg"):
    image_file_names.append(file)

for image_name in image_file_names:
    if image_name[3] !='.':
        print ("Error: Not as per required format: Check first 3 digits for this image name:\n", image_name)
        if not bool(re.match('^[0-9]+$', image_name[0:3])):
            print ("Error: Not as per required format for this image:\n", image_name)


In [4]:
#start label img
# !important: while labelling ensure that labels are numerical, starting from 100. 100,101,102,103,..... etc.
#as the code is designed to handle only the numerals.
os.system("labelimg")

Image:/Users/ml/Desktop/ILS/object_detection/image_preprocessing/output_data/100.jpg -> Annotation:/Users/ml/Desktop/ILS/object_detection/image_preprocessing/output_data/100.xml
Image:/Users/ml/Desktop/ILS/object_detection/image_preprocessing/output_data/101.jpg -> Annotation:/Users/ml/Desktop/ILS/object_detection/image_preprocessing/output_data/101.xml
Image:/Users/ml/Desktop/ILS/object_detection/image_preprocessing/output_data/102.jpg -> Annotation:/Users/ml/Desktop/ILS/object_detection/image_preprocessing/output_data/102.xml
Image:/Users/ml/Desktop/ILS/object_detection/image_preprocessing/output_data/103.jpg -> Annotation:/Users/ml/Desktop/ILS/object_detection/image_preprocessing/output_data/103.xml
Image:/Users/ml/Desktop/ILS/object_detection/image_preprocessing/output_data/104.jpg -> Annotation:/Users/ml/Desktop/ILS/object_detection/image_preprocessing/output_data/104.xml
Image:/Users/ml/Desktop/ILS/object_detection/image_preprocessing/output_data/105.jpg -> Annotation:/Users/ml/D

0

In [7]:
#get all the xml file names generated after labelling from previous step
file_names_xml = []
#data_dir = "..//output_data/"
os.chdir(data_dir)
for file in glob.glob("*.xml"):
    file_names_xml.append(file)

In [8]:
#ensure number of images equals number of xml and then organize file name in ascending order

if len(file_names_xml) != len(image_file_names):
    print("Error: Inconsistent number of xml and image files")

file_names_sorted = []
for file in file_names_xml:
    file_names_sorted.append(int(file[0:3]))

file_names_sorted.sort()
file_names = [str(x)+'.xml' for x in file_names_sorted]

In [9]:
#convert xml file to format to be consumed by retina net algorithm

bbox_formatted_all = []
name_obj_all = []
for file_name in file_names:
    
    tree = ET.parse(file_name)
    
    root = tree.getroot()

    bbox_formatted= []
    name_obj_formatted= []
    obj_bbox = root.findall('./object')
    name_obj = []
    for item in obj_bbox:
        bbox = []
        name_obj.append(item.find('name').text)
        bbox_cord= item.find('./bndbox')
        bbox.append(
            [
            bbox_cord.find("ymin").text,
            bbox_cord.find("xmin").text,
            bbox_cord.find("ymax").text,
            bbox_cord.find("xmax").text
            ]  
        )
        bbox_formatted.append((bbox[0]))
    bbox_formatted_all.append(np.array(bbox_formatted, dtype = np.float32)/640.0) #640 is both and width dimention.
    name_obj_all.append(np.array(name_obj).astype(int))


In [10]:
#save bbox as .npy file
bbox_fn = 'bbox.npy'
with open(bbox_fn, 'wb') as f:
    np.save(f, bbox_formatted_all)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/lib/npyio.py:521: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [11]:
#save index of the class list
indices_class_fn = 'indices_class_list.npy'

name_obj_all = np.array(name_obj_all).astype(object)
with open(indices_class_fn, 'wb') as f:
    np.save(f, name_obj_all)

/var/folders/58/gd1m2zgn4hzgrrx4xcds_bg80000gp/T/ipykernel_3770/3861693921.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  name_obj_all = np.array(name_obj_all).astype(object)


In [4]:
#class index dictionary creation:
import pickle
#define category index
num_classes = 3
input_field_id = 1
dropdown_id = 2
text_id = 3

category_index = {
    input_field_id: {'id': input_field_id, 'name': 'Input Field'},
    dropdown_id:{'id':dropdown_id, 'name':'Drop Down'},
    text_id:{'id':text_id, 'name':'Text'}
    }

file_path = "../data/category_index.pickle"
with open(file_path, 'wb') as file:
    pickle.dump(category_index, file)
